# Loop Order Prediction and Misclassification Analysis

This notebook trains an XGBoost model to predict loop order (COEFFICIENTS) using graph features from loops 5-10, and then analyzes misclassifications in the training set.

## Data Overview
- **Training data**: 5loopfeats.csv, 6loopfeats.csv, 7loopfeats.csv, 8loopfeats.csv, 9loopfeats.csv, 10loopfeats.csv
- **Test data**: 11loopfeats.csv
- **Target variable**: COEFFICIENTS (first column)
- **Features**: Graph structural features (Basic, Connectivity, Centrality, Core, Robust, Cycle, Spectral, Kirchhoff, Planarity, Symmetry)


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [39]:
# Define data path
data_path = "/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular"

# Load training data (loops 5-10)
training_files = ['9loopfeats_enhanced.csv']
training_data = []

print("Loading training data...")
for file in training_files:
    print(f"Loading {file}...")
    df = pd.read_csv(f"{data_path}/{file}")
    df['source_loop'] = file.replace('loopfeats.csv', '')  # Add source loop identifier
    training_data.append(df)
    print(f"  Shape: {df.shape}")

# Combine all training data
train_df = pd.concat(training_data, ignore_index=True)
print(f"\nCombined training data shape: {train_df.shape}")
print(f"Target variable (COEFFICIENTS) unique values: {sorted(train_df['COEFFICIENTS'].unique())}")
print(f"Target variable distribution:")
print(train_df['COEFFICIENTS'].value_counts().sort_index())


Loading training data...
Loading 9loopfeats_enhanced.csv...
  Shape: (13972, 209)

Combined training data shape: (13972, 209)
Target variable (COEFFICIENTS) unique values: [np.int64(0), np.int64(1)]
Target variable distribution:
COEFFICIENTS
0    8311
1    5661
Name: count, dtype: int64


In [40]:
train_df.shape

(13972, 209)

In [43]:
# Load test data (loop 11)
print("Loading test data...")
test_df = pd.read_csv(f"{data_path}/9loopfeats_enhanced.csv")
test_df['source_loop'] = '9'  # Add source loop identifier
print(f"Test data shape: {test_df.shape}")
print(f"Test target variable (COEFFICIENTS) unique values: {sorted(test_df['COEFFICIENTS'].unique())}")
print(f"Test target variable distribution:")
print(test_df['COEFFICIENTS'].value_counts().sort_index())


Loading test data...
Test data shape: (13972, 209)
Test target variable (COEFFICIENTS) unique values: [np.int64(0), np.int64(1)]
Test target variable distribution:
COEFFICIENTS
0    8311
1    5661
Name: count, dtype: int64


In [44]:
# Prepare features and target
# Remove non-feature columns
feature_columns = [col for col in train_df.columns if col not in ['COEFFICIENTS', 'source_loop']]

X_train = train_df[feature_columns]
y_train = train_df['COEFFICIENTS']
X_test = test_df[feature_columns]
y_test = test_df['COEFFICIENTS']

print(f"Number of features: {len(feature_columns)}")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Check for missing values
print(f"\nMissing values in training set: {X_train.isnull().sum().sum()}")
print(f"Missing values in test set: {X_test.isnull().sum().sum()}")

# Handle any missing values by filling with median


Number of features: 207
Training set: 13972 samples
Test set: 13972 samples

Missing values in training set: 754488
Missing values in test set: 754488


In [45]:
# question - are there more nulls at 11 loops for this kind of model?

In [46]:
X_train.shape

(13972, 207)

In [47]:
X_test

,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,Basic_degree_entropy,...,RandomWalk_commute_time_variance,RandomWalk_spectral_gap,RandomWalk_relaxation_time,RandomWalk_cutoff_time,Spectral_adjacency_energy,Spectral_laplacian_energy,Spectral_signless_laplacian_energy,Spectral_normalized_laplacian_energy,Spectral_incidence_energy,Spectral_distance_energy
0,13,33,4,6,5.076923,0.474186,0.230525,0.423077,2.538462,0.991264,...,NaN,2.243814,0.445670,1.143120,NaN,NaN,NaN,NaN,NaN,NaN
1,13,32,4,6,4.923077,0.474186,-0.230525,0.410256,2.461538,0.991264,...,NaN,2.195224,0.455534,1.168423,NaN,NaN,NaN,NaN,NaN,NaN
2,13,31,4,5,4.769231,0.421325,-1.278019,0.397436,2.384615,0.779350,...,NaN,2.015539,0.496145,1.272588,NaN,NaN,NaN,NaN,NaN,NaN
3,13,33,4,6,5.076923,0.615385,-0.046875,0.423077,2.538462,1.334679,...,NaN,2.055523,0.486494,1.247833,NaN,NaN,NaN,NaN,NaN,NaN
4,13,32,4,6,4.923077,0.474186,-0.230525,0.410256,2.461538,0.991264,...,NaN,2.024193,0.494024,1.267147,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13967,13,32,4,6,4.923077,0.916644,0.152471,0.410256,2.461538,1.460485,...,NaN,1.024267,0.976308,2.504180,NaN,NaN,NaN,NaN,NaN,NaN
13968,13,30,4,6,4.615385,0.923077,0.833333,0.384615,2.307692,0.890492,...,NaN,0.648907,1.541054,3.952725,NaN,NaN,NaN,NaN,NaN,NaN
13969,13,28,4,5,4.307692,0.461538,0.833333,0.358974,2.153846,0.890492,...,NaN,0.286415,3.491433,8.955349,NaN,NaN,NaN,NaN,NaN,NaN
13970,13,33,4,7,5.076923,1.071414,0.222049,0.423077,2.538462,1.614331,...,NaN,1.064242,0.939636,2.410117,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# Train XGBoost model
from xgboost import XGBClassifier
print("Training XGBoost model...")

# Set up XGBoost parameters
#xgb_params = {
#    'objective': 'multi:softmax',  # Multi-class classification
#    'num_class': len(y_train.unique()),
#    'max_depth': 6,
#    'learning_rate': 0.1,
#    'n_estimators': 100,
#    'subsample': 0.8,
#    'colsample_bytree': 0.8,
#    'random_state': 42,
#    'n_jobs': -1
#}

# Create and train the model

model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)

#model = xgb.XGBClassifier(**xgb_params)
model.fit(X_train, y_train)

print("Model training completed!")


Training XGBoost model...
Model training completed!


In [49]:
from sklearn.metrics import roc_auc_score

In [50]:
# Make predictions on training and test sets
print("Making predictions...")

# Training set predictions
y_train_pred = model.predict_proba(X_train)[:,1]
train_accuracy = roc_auc_score(y_train, y_train_pred)

# Test set predictions
y_test_pred = model.predict_proba(X_test)[:,1]
test_accuracy = roc_auc_score(y_test, y_test_pred)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

# Add predictions to dataframes for analysis
train_df['predicted'] = y_train_pred
train_df['correct'] = (train_df['COEFFICIENTS'] == train_df['predicted'])
test_df['predicted'] = y_test_pred
test_df['correct'] = (test_df['COEFFICIENTS'] == test_df['predicted'])


Making predictions...
Training accuracy: 0.9999
Test accuracy: 0.9999


In [38]:
# Make predictions on training and test sets
print("Making predictions...")

# Training set predictions
y_train_pred = model.predict_proba(X_train)[:,1]
train_accuracy = roc_auc_score(y_train, y_train_pred)

# Test set predictions
y_test_pred = model.predict_proba(X_test)[:,1]
test_accuracy = roc_auc_score(y_test, y_test_pred)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

# Add predictions to dataframes for analysis
train_df['predicted'] = y_train_pred
train_df['correct'] = (train_df['COEFFICIENTS'] == train_df['predicted'])
test_df['predicted'] = y_test_pred
test_df['correct'] = (test_df['COEFFICIENTS'] == test_df['predicted'])


Making predictions...
Training accuracy: 0.9978
Test accuracy: 0.7987


In [28]:
train_df.to_csv("/Users/rezadoobary/Documents/ML-correlator/Misclassifications/data/train_tree.csv")
test_df.to_csv("/Users/rezadoobary/Documents/ML-correlator/Misclassifications/data/test_tree.csv")

## Misclassification Analysis

Now let's analyze the misclassifications in the training set to understand where the model struggles.


In [52]:
# how to choose threshold?

In [51]:
# Overall misclassification statistics
misclassified = train_df[~train_df['correct']]
total_samples = len(train_df)
misclassified_count = len(misclassified)

print(f"Total training samples: {total_samples}")
print(f"Misclassified samples: {misclassified_count}")
print(f"Misclassification rate: {misclassified_count/total_samples:.4f}")

# Misclassification by source loop
print("\nMisclassification by source loop:")
misclass_by_loop = train_df.groupby('source_loop').agg({
    'correct': ['count', 'sum', lambda x: (x == False).sum()]
}).round(4)
misclass_by_loop.columns = ['Total', 'Correct', 'Misclassified']
misclass_by_loop['Misclass_Rate'] = misclass_by_loop['Misclassified'] / misclass_by_loop['Total']
print(misclass_by_loop)


Total training samples: 13972
Misclassified samples: 13972
Misclassification rate: 1.0000

Misclassification by source loop:
                         Total  Correct  Misclassified  Misclass_Rate
source_loop                                                          
9loopfeats_enhanced.csv  13972        0          13972            1.0


In [9]:
misclassified

,COEFFICIENTS,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,...,RandomWalk_return_probability,RandomWalk_escape_probability,RandomWalk_centrality_entropy,Embedding_planar_embedding_quality,Embedding_face_degree_distribution_mean,Embedding_face_degree_distribution_std,Embedding_dual_graph_features,source_loop,predicted,correct
49,0,13,30,4,6,4.615385,0.737820,0.747932,0.384615,2.307692,...,0.076923,0.923077,6.803303,1.0,3.157895,0.364642,19,9loopfeats_enhanced.csv,1,False
52,0,13,30,4,6,4.615385,0.624926,0.503556,0.384615,2.307692,...,0.076923,0.923077,6.827511,1.0,3.157895,0.364642,19,9loopfeats_enhanced.csv,1,False
56,0,13,31,4,6,4.769231,0.799408,0.438359,0.397436,2.384615,...,0.076923,0.923077,6.774982,1.0,3.100000,0.300000,20,9loopfeats_enhanced.csv,1,False
71,1,13,31,4,6,4.769231,0.696568,0.347455,0.397436,2.384615,...,0.076923,0.923077,6.799190,1.0,3.100000,0.300000,20,9loopfeats_enhanced.csv,0,False
75,0,13,31,4,6,4.769231,0.696568,0.347455,0.397436,2.384615,...,0.076923,0.923077,6.799190,1.0,3.100000,0.300000,20,9loopfeats_enhanced.csv,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13643,1,13,28,4,5,4.307692,0.461538,0.833333,0.358974,2.153846,...,0.076923,0.923077,6.859945,1.0,3.294118,0.570315,17,9loopfeats_enhanced.csv,0,False
13650,1,13,30,4,6,4.615385,0.737820,0.747932,0.384615,2.307692,...,0.076923,0.923077,6.803303,1.0,3.157895,0.364642,19,9loopfeats_enhanced.csv,0,False
13686,1,13,31,4,7,4.769231,0.890449,1.121740,0.397436,2.384615,...,0.076923,0.923077,6.754851,1.0,3.100000,0.300000,20,9loopfeats_enhanced.csv,0,False
13693,1,13,29,4,6,4.461538,0.634324,1.048610,0.371795,2.230769,...,0.076923,0.923077,6.831624,1.0,3.222222,0.415740,18,9loopfeats_enhanced.csv,0,False


In [8]:
# Display the misclassified samples dataframe with all features
print("Misclassified Samples DataFrame:")
print("=" * 50)

# Show the first 20 misclassified samples with all their features
print(f"Showing first 20 out of {len(misclassified)} misclassified samples:")
print(f"Total columns: {len(misclassified.columns)}")

# Display the dataframe
display(misclassified.head(20))

# Also show some basic info about the misclassified samples
print(f"\nMisclassified samples info:")
print(f"Shape: {misclassified.shape}")
print(f"Columns: {list(misclassified.columns)}")
print(f"Data types:\n{misclassified.dtypes}")

Misclassified Samples DataFrame:
Showing first 20 out of 1253 misclassified samples:
Total columns: 60


,COEFFICIENTS,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,...,Kirchhoff_index,Planarity_num_faces,Planarity_face_size_mean,Planarity_face_size_max,Symmetry_automorphism_group_order,Symmetry_num_orbits,Symmetry_orbit_size_max,source_loop,predicted,correct
51,0,13,30,4,6,4.615385,0.737820,0.747932,0.384615,2.307692,...,38.792243,19,3.157895,4,1,13,1,9,1,False
56,0,13,31,4,6,4.769231,0.799408,0.438359,0.397436,2.384615,...,37.321277,20,3.100000,4,1,13,1,9,1,False
71,1,13,31,4,6,4.769231,0.696568,0.347455,0.397436,2.384615,...,37.346345,20,3.100000,4,1,13,1,9,0,False
75,0,13,31,4,6,4.769231,0.696568,0.347455,0.397436,2.384615,...,37.342107,20,3.100000,4,1,13,1,9,1,False
85,0,13,31,4,7,4.769231,0.890449,1.121740,0.397436,2.384615,...,37.266592,20,3.100000,4,1,13,1,9,1,False
87,0,13,30,4,7,4.615385,0.835598,1.610225,0.384615,2.307692,...,38.766478,19,3.157895,4,1,13,1,9,1,False
88,0,13,29,4,6,4.461538,0.634324,1.048610,0.371795,2.230769,...,40.212789,18,3.222222,4,2,7,2,9,1,False
92,0,13,31,4,6,4.769231,0.799408,0.438359,0.397436,2.384615,...,37.485907,20,3.100000,5,2,8,2,9,1,False
99,0,13,31,4,6,4.769231,0.799408,0.438359,0.397436,2.384615,...,37.213093,20,3.100000,4,1,13,1,9,1,False
106,0,13,31,4,6,4.769231,0.799408,0.438359,0.397436,2.384615,...,37.363579,20,3.100000,4,1,13,1,9,1,False



Misclassified samples info:
Shape: (1253, 60)
Columns: ['COEFFICIENTS', 'Basic_num_nodes', 'Basic_num_edges', 'Basic_min_degree', 'Basic_max_degree', 'Basic_avg_degree', 'Basic_degree_std', 'Basic_degree_skew', 'Basic_density', 'Basic_edge_to_node_ratio', 'Basic_degree_entropy', 'Connectivity_is_connected', 'Connectivity_num_components', 'Connectivity_diameter', 'Connectivity_radius', 'Connectivity_avg_shortest_path_length', 'Connectivity_wiener_index', 'Centrality_betweenness_mean', 'Centrality_betweenness_max', 'Centrality_betweenness_std', 'Centrality_betweenness_skew', 'Centrality_closeness_mean', 'Centrality_closeness_max', 'Centrality_closeness_std', 'Centrality_closeness_skew', 'Centrality_eigenvector_mean', 'Centrality_eigenvector_max', 'Centrality_eigenvector_std', 'Centrality_eigenvector_skew', 'Core_max_core_index', 'Core_core_index_mean', 'Robust_articulation_points', 'Robust_bridge_count', 'Cycle_num_cycles_len_5', 'Cycle_num_cycles_len_6', 'Spectral_algebraic_connectivit

In [31]:
# Analysis of Misclassified Cases with Identical Feature Spaces

# Extract feature columns (excluding target, predictions, and metadata)
feature_cols = [col for col in misclassified.columns if col not in ['COEFFICIENTS', 'predicted', 'correct', 'source_loop']]
misclassified_features = misclassified[feature_cols].copy()

print(f"Analyzing {len(misclassified)} misclassified cases with {len(feature_cols)} features each")
print("=" * 80)

# Find cases with identical feature spaces
print("1. Finding cases with EXACTLY identical feature spaces...")
print("-" * 50)

# Group by all feature values to find identical cases
identical_groups = misclassified_features.groupby(feature_cols).size().reset_index(name='count')
identical_groups = identical_groups[identical_groups['count'] > 1].sort_values('count', ascending=False)

print(f"Found {len(identical_groups)} groups of cases with identical feature spaces:")
print(f"Total cases in identical groups: {identical_groups['count'].sum()}")

if len(identical_groups) > 0:
    print(f"\nTop 10 groups with most identical cases:")
    for i, (idx, row) in enumerate(identical_groups.head(10).iterrows()):
        print(f"  Group {i+1}: {row['count']} cases with identical features")
        
    # Show details of the largest group
    largest_group_size = identical_groups.iloc[0]['count']
    print(f"\nDetails of largest group ({largest_group_size} cases):")
    
    # Find the actual cases in the largest group
    largest_group_features = identical_groups.iloc[0][feature_cols]
    mask = (misclassified_features == largest_group_features).all(axis=1)
    largest_group_cases = misclassified[mask]
    
    print(f"Actual vs Predicted labels in largest group:")
    print(largest_group_cases[['COEFFICIENTS', 'predicted']].value_counts().sort_index())
else:
    print("No cases found with exactly identical feature spaces.")

print("\n" + "=" * 80)

# Analyze feature similarity percentages
print("2. Analyzing percentage of identical features between all pairs...")
print("-" * 50)

# Sample a subset for computational efficiency (too many pairs otherwise)
sample_size = min(100, len(misclassified_features))
print(f"Sampling {sample_size} cases for pairwise similarity analysis...")

# Random sample for analysis
np.random.seed(42)
sample_indices = np.random.choice(len(misclassified_features), sample_size, replace=False)
sample_features = misclassified_features.iloc[sample_indices]

# Calculate pairwise similarities
similarities = []
for i in range(len(sample_features)):
    for j in range(i+1, len(sample_features)):
        case1 = sample_features.iloc[i]
        case2 = sample_features.iloc[j]
        
        # Calculate percentage of identical features
        identical_features = (case1 == case2).sum()
        similarity_percentage = (identical_features / len(feature_cols)) * 100
        
        similarities.append({
            'case1_idx': sample_indices[i],
            'case2_idx': sample_indices[j],
            'identical_features': identical_features,
            'similarity_percentage': similarity_percentage
        })

similarities_df = pd.DataFrame(similarities)

print(f"Analyzed {len(similarities)} pairs of misclassified cases")
print(f"Similarity statistics:")
print(f"  Mean similarity: {similarities_df['similarity_percentage'].mean():.2f}%")
print(f"  Median similarity: {similarities_df['similarity_percentage'].median():.2f}%")
print(f"  Max similarity: {similarities_df['similarity_percentage'].max():.2f}%")
print(f"  Min similarity: {similarities_df['similarity_percentage'].min():.2f}%")

# Group by similarity ranges
print(f"\nSimilarity distribution:")
similarity_ranges = [
    (100, 100, "Exactly identical"),
    (90, 99, "90-99% identical"),
    (80, 89, "80-89% identical"),
    (70, 79, "70-79% identical"),
    (60, 69, "60-69% identical"),
    (50, 59, "50-59% identical"),
    (0, 49, "Less than 50% identical")
]

for min_sim, max_sim, label in similarity_ranges:
    count = len(similarities_df[
        (similarities_df['similarity_percentage'] >= min_sim) & 
        (similarities_df['similarity_percentage'] <= max_sim)
    ])
    percentage = (count / len(similarities_df)) * 100
    print(f"  {label}: {count} pairs ({percentage:.1f}%)")

# Show most similar pairs
print(f"\nTop 10 most similar pairs:")
top_similar = similarities_df.nlargest(10, 'similarity_percentage')
for i, (idx, row) in enumerate(top_similar.iterrows()):
    print(f"  {i+1}. Cases {row['case1_idx']} & {row['case2_idx']}: "
          f"{row['identical_features']}/{len(feature_cols)} features identical "
          f"({row['similarity_percentage']:.1f}%)")

print("\n" + "=" * 80)

# Feature-wise analysis for most common identical features
print("3. Feature-wise analysis: Which features are most commonly identical?")
print("-" * 50)

# For the sample, count how often each feature has identical values
feature_identity_counts = {}
for feature in feature_cols:
    # Count how many pairs have identical values for this feature
    identical_count = 0
    total_pairs = len(similarities_df)
    
    for _, row in similarities_df.iterrows():
        # Get the actual indices in the sample_features dataframe
        case1_sample_idx = np.where(sample_indices == row['case1_idx'])[0][0]
        case2_sample_idx = np.where(sample_indices == row['case2_idx'])[0][0]
        
        case1_val = sample_features.iloc[case1_sample_idx][feature]
        case2_val = sample_features.iloc[case2_sample_idx][feature]
        if case1_val == case2_val:
            identical_count += 1
    
    feature_identity_counts[feature] = (identical_count / total_pairs) * 100

# Sort by most commonly identical
feature_identity_df = pd.DataFrame([
    {'feature': feature, 'identical_percentage': percentage}
    for feature, percentage in feature_identity_counts.items()
]).sort_values('identical_percentage', ascending=False)

print("Top 15 features most commonly identical between misclassified pairs:")
for i, (idx, row) in enumerate(feature_identity_df.head(60).iterrows()):
    print(f"  {i+1:2d}. {row['feature']:<35}: {row['identical_percentage']:5.1f}%")

print("\n" + "=" * 80)
print("SUMMARY:")
print(f"- {len(identical_groups)} groups of cases with exactly identical feature spaces")
print(f"- {identical_groups['count'].sum() if len(identical_groups) > 0 else 0} total cases in identical groups")
print(f"- Average similarity between random pairs: {similarities_df['similarity_percentage'].mean():.1f}%")
print(f"- Most similar pair: {similarities_df['similarity_percentage'].max():.1f}% identical features")

Analyzing 1253 misclassified cases with 56 features each
1. Finding cases with EXACTLY identical feature spaces...
--------------------------------------------------
Found 0 groups of cases with identical feature spaces:
Total cases in identical groups: 0
No cases found with exactly identical feature spaces.

2. Analyzing percentage of identical features between all pairs...
--------------------------------------------------
Sampling 100 cases for pairwise similarity analysis...
Analyzed 4950 pairs of misclassified cases
Similarity statistics:
  Mean similarity: 30.93%
  Median similarity: 28.57%
  Max similarity: 58.93%
  Min similarity: 16.07%

Similarity distribution:
  Exactly identical: 0 pairs (0.0%)
  90-99% identical: 0 pairs (0.0%)
  80-89% identical: 0 pairs (0.0%)
  70-79% identical: 0 pairs (0.0%)
  60-69% identical: 0 pairs (0.0%)
  50-59% identical: 78 pairs (1.6%)
  Less than 50% identical: 4872 pairs (98.4%)

Top 10 most similar pairs:
  1. Cases 624.0 & 620.0: 33.0/56 

# Additional Graph Features for Better Separation

Based on your current feature set and the analysis showing that misclassified cases have relatively low similarity (30.93% average), here are additional graph features that could help better separate your graphs:

## Current Features Analysis
Your current features cover:
- **Basic**: nodes, edges, degree statistics, density
- **Connectivity**: connectedness, components, diameter, radius, shortest paths
- **Centrality**: betweenness, closeness, eigenvector (mean, max, std, skew)
- **Core**: core decomposition
- **Robustness**: articulation points, bridges
- **Cycles**: cycles of length 5 and 6
- **Spectral**: algebraic connectivity, spectral gap, Laplacian eigenvalues
- **Kirchhoff**: resistance distance
- **Planarity**: face count and sizes
- **Symmetry**: automorphism group properties

## Suggested Additional Features

### 1. **Higher-Order Cycle Features**
```python
# Cycles of different lengths (3, 4, 7, 8, 9, 10)
- Cycle_num_cycles_len_3
- Cycle_num_cycles_len_4  
- Cycle_num_cycles_len_7
- Cycle_num_cycles_len_8
- Cycle_num_cycles_len_9
- Cycle_num_cycles_len_10
```

### 2. **Advanced Spectral Features**
```python
# More Laplacian eigenvalues (you only have 0-9)
- Spectral_lap_eig_10 to Spectral_lap_eig_20
- Spectral_second_smallest_eigenvalue
- Spectral_largest_eigenvalue
- Spectral_eigenvalue_gap_ratio
- Spectral_energy (sum of eigenvalue squares)
```

### 3. **Subgraph Pattern Features**
```python
# Count specific subgraph patterns
- Subgraph_triangle_count
- Subgraph_square_count  
- Subgraph_pentagon_count
- Subgraph_hexagon_count
- Subgraph_star_count (k-star patterns)
- Subgraph_path_count (paths of different lengths)
- Subgraph_clique_count (max clique size)
```

### 4. **Advanced Centrality Measures**
```python
# Additional centrality measures
- Centrality_harmonic_mean
- Centrality_katz_centrality
- Centrality_load_centrality
- Centrality_percolation_centrality
- Centrality_subgraph_centrality
```

### 5. **Graph Distance and Similarity Features**
```python
# Distance-based features
- Distance_eccentricity_mean
- Distance_eccentricity_max
- Distance_eccentricity_std
- Distance_radius_to_diameter_ratio
- Distance_avg_path_length_to_diameter_ratio
```

### 6. **Connectivity Robustness Features**
```python
# More robustness measures
- Robust_node_connectivity
- Robust_edge_connectivity
- Robust_minimum_cut_size
- Robust_connectivity_ratio
- Robust_failure_tolerance
```

### 7. **Planar Graph Specific Features**
```python
# Enhanced planar features
- Planarity_dual_graph_features
- Planarity_face_degree_distribution
- Planarity_outer_face_size
- Planarity_inner_faces_count
- Planarity_planar_embedding_quality
```

### 8. **Topological Features**
```python
# Topological invariants
- Topology_genus
- Topology_chromatic_number
- Topology_independence_number
- Topology_dominating_number
- Topology_matching_number
```

### 9. **Random Walk Features**
```python
# Random walk properties
- RandomWalk_hitting_time_mean
- RandomWalk_cover_time
- RandomWalk_mixing_time
- RandomWalk_stationary_distribution_entropy
```

### 10. **Graph Polynomial Features**
```python
# Graph polynomials and invariants
- Polynomial_characteristic_polynomial_coeffs
- Polynomial_chromatic_polynomial_coeffs
- Polynomial_tutte_polynomial_coeffs
```

## Implementation Priority

**High Priority** (likely to have immediate impact):
1. Higher-order cycle features (3, 4, 7, 8, 9, 10)
2. More Laplacian eigenvalues (10-20)
3. Subgraph pattern counts (triangles, squares, etc.)
4. Advanced centrality measures

**Medium Priority**:
5. Graph distance features
6. Connectivity robustness measures
7. Random walk features

**Lower Priority** (computationally expensive):
8. Topological features
9. Graph polynomial features
10. Enhanced planar features

## Why These Features Help

1. **Cycle features**: Different loop orders likely have different cycle distributions
2. **Spectral features**: More eigenvalues capture finer structural details
3. **Subgraph patterns**: Capture local structural motifs that distinguish graphs
4. **Centrality measures**: Different centrality measures capture different aspects of node importance
5. **Robustness features**: Measure how graphs respond to failures/removals

These features should help create more distinctive "fingerprints" for each graph, reducing the similarity between misclassified cases and improving classification accuracy.


In [32]:
# Example Implementation of High-Priority Additional Features

import networkx as nx
import numpy as np
from collections import Counter

def compute_additional_graph_features(graph_edges):
    """
    Compute additional graph features that could help better separate graphs.
    
    Args:
        graph_edges: List of edges [(u1, v1), (u2, v2), ...]
    
    Returns:
        dict: Dictionary of additional features
    """
    # Build NetworkX graph
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    
    features = {}
    
    # 1. Higher-order cycle features
    print("Computing cycle features...")
    try:
        # Count cycles of different lengths
        cycles_3 = len([c for c in nx.simple_cycles(nx.DiGraph(G)) if len(c) == 3])
        cycles_4 = len([c for c in nx.simple_cycles(nx.DiGraph(G)) if len(c) == 4])
        
        # For undirected graphs, we need to be more careful
        # Count triangles (3-cycles)
        triangles = sum(nx.triangles(G).values()) // 3
        features['Cycle_triangles'] = triangles
        
        # Count squares (4-cycles) - approximate
        squares = 0
        for node in G.nodes():
            neighbors = list(G.neighbors(node))
            for i in range(len(neighbors)):
                for j in range(i+1, len(neighbors)):
                    if G.has_edge(neighbors[i], neighbors[j]):
                        squares += 1
        features['Cycle_squares'] = squares // 4
        
    except Exception as e:
        print(f"Error computing cycles: {e}")
        features['Cycle_triangles'] = 0
        features['Cycle_squares'] = 0
    
    # 2. Advanced spectral features
    print("Computing spectral features...")
    try:
        # Laplacian matrix
        L = nx.laplacian_matrix(G).astype(float)
        eigenvals = np.linalg.eigvals(L.toarray())
        eigenvals = np.sort(eigenvals)
        
        # More eigenvalues (if graph is large enough)
        n_nodes = G.number_of_nodes()
        if n_nodes >= 20:
            features['Spectral_lap_eig_10'] = eigenvals[10] if len(eigenvals) > 10 else 0
            features['Spectral_lap_eig_11'] = eigenvals[11] if len(eigenvals) > 11 else 0
            features['Spectral_lap_eig_12'] = eigenvals[12] if len(eigenvals) > 12 else 0
        
        # Spectral energy
        features['Spectral_energy'] = np.sum(eigenvals**2)
        
        # Eigenvalue gap ratios
        if len(eigenvals) > 1:
            features['Spectral_gap_ratio'] = (eigenvals[1] - eigenvals[0]) / eigenvals[0] if eigenvals[0] != 0 else 0
        
    except Exception as e:
        print(f"Error computing spectral features: {e}")
        features['Spectral_energy'] = 0
        features['Spectral_gap_ratio'] = 0
    
    # 3. Subgraph pattern features
    print("Computing subgraph patterns...")
    try:
        # Clique number (maximum clique size)
        features['Subgraph_max_clique'] = nx.graph_clique_number(G)
        
        # Independence number
        features['Subgraph_independence_number'] = nx.graph_number_of_cliques(G, 1)
        
        # Path counts of different lengths
        path_counts = {}
        for length in [3, 4, 5]:
            count = 0
            for node in G.nodes():
                for path in nx.all_simple_paths(G, node, cutoff=length):
                    if len(path) == length + 1:
                        count += 1
            path_counts[f'Subgraph_path_len_{length}'] = count // 2  # Divide by 2 for undirected
        
        features.update(path_counts)
        
    except Exception as e:
        print(f"Error computing subgraph patterns: {e}")
        features['Subgraph_max_clique'] = 0
        features['Subgraph_independence_number'] = 0
    
    # 4. Advanced centrality measures
    print("Computing advanced centrality...")
    try:
        # Katz centrality
        katz = nx.katz_centrality(G, alpha=0.1)
        features['Centrality_katz_mean'] = np.mean(list(katz.values()))
        features['Centrality_katz_max'] = np.max(list(katz.values()))
        features['Centrality_katz_std'] = np.std(list(katz.values()))
        
        # Harmonic centrality
        harmonic = nx.harmonic_centrality(G)
        features['Centrality_harmonic_mean'] = np.mean(list(harmonic.values()))
        features['Centrality_harmonic_max'] = np.max(list(harmonic.values()))
        
    except Exception as e:
        print(f"Error computing advanced centrality: {e}")
        features['Centrality_katz_mean'] = 0
        features['Centrality_harmonic_mean'] = 0
    
    # 5. Graph distance features
    print("Computing distance features...")
    try:
        # Eccentricity statistics
        eccentricity = nx.eccentricity(G)
        ecc_values = list(eccentricity.values())
        features['Distance_eccentricity_mean'] = np.mean(ecc_values)
        features['Distance_eccentricity_max'] = np.max(ecc_values)
        features['Distance_eccentricity_std'] = np.std(ecc_values)
        
        # Radius to diameter ratio
        radius = nx.radius(G)
        diameter = nx.diameter(G)
        features['Distance_radius_to_diameter_ratio'] = radius / diameter if diameter > 0 else 0
        
    except Exception as e:
        print(f"Error computing distance features: {e}")
        features['Distance_eccentricity_mean'] = 0
        features['Distance_radius_to_diameter_ratio'] = 0
    
    # 6. Connectivity robustness features
    print("Computing robustness features...")
    try:
        # Node connectivity
        features['Robust_node_connectivity'] = nx.node_connectivity(G)
        
        # Edge connectivity
        features['Robust_edge_connectivity'] = nx.edge_connectivity(G)
        
        # Connectivity ratio
        features['Robust_connectivity_ratio'] = features['Robust_node_connectivity'] / features['Robust_edge_connectivity'] if features['Robust_edge_connectivity'] > 0 else 0
        
    except Exception as e:
        print(f"Error computing robustness features: {e}")
        features['Robust_node_connectivity'] = 0
        features['Robust_edge_connectivity'] = 0
    
    return features

# Example usage with a sample graph
print("Example: Computing additional features for a sample graph...")
sample_edges = [(0, 1), (1, 2), (2, 3), (3, 0), (1, 3)]  # A simple graph
additional_features = compute_additional_graph_features(sample_edges)

print("\nAdditional features computed:")
for feature, value in additional_features.items():
    print(f"  {feature}: {value}")

print(f"\nTotal additional features: {len(additional_features)}")
print("These features could be added to your existing feature set to improve graph separation.")


Example: Computing additional features for a sample graph...
Computing cycle features...
Computing spectral features...
Computing subgraph patterns...
Error computing subgraph patterns: object of type 'int' has no len()
Computing advanced centrality...
Computing distance features...
Computing robustness features...

Additional features computed:
  Cycle_triangles: 2
  Cycle_squares: 1
  Spectral_energy: 35.999999999999986
  Spectral_gap_ratio: 0
  Subgraph_max_clique: 0
  Subgraph_independence_number: 0
  Centrality_katz_mean: 0.49952807931359733
  Centrality_katz_max: 0.5212466883221977
  Centrality_katz_std: 0.02171860900860037
  Centrality_harmonic_mean: 2.75
  Centrality_harmonic_max: 3.0
  Distance_eccentricity_mean: 1.5
  Distance_eccentricity_max: 2
  Distance_eccentricity_std: 0.5
  Distance_radius_to_diameter_ratio: 0.5
  Robust_node_connectivity: 2
  Robust_edge_connectivity: 2
  Robust_connectivity_ratio: 1.0

Total additional features: 18
These features could be added to yo

In [28]:
misclassified['Symmetry_num_orbits'].unique()

array([13,  7,  8,  9, 10])

In [29]:
train_df['Symmetry_num_orbits'].unique()

array([ 6,  8,  7, 13,  9,  5, 10,  4,  2, 12, 11])

In [30]:
train_df.shape

(13972, 60)